# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /home/ish/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ish/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
from sklearn.externals import joblib
import re

from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize

from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier

/home/ish/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/ish/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
/home/ish/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests i

In [3]:
# load data from database
engine = create_engine('sqlite:///../data/DisasterResponse.db')
df = pd.read_sql('SELECT * FROM disaster_messages_clean',engine)
X = df.message
y = df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(sentence):
    '''
    tokenizes a sentence after normalizing it and returns lemmatized tokens.
    '''
    # normalizing, tokenizing, lemmatizing the sentence
    sentence = re.sub('\W',' ',sentence)
    tokens = word_tokenize(sentence.lower().strip())
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(i,'n') for i in tokens]
    tokens = [lemmatizer.lemmatize(i,'v') for i in tokens]
    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
pipeline = Pipeline([
        ('vect',CountVectorizer(tokenizer = tokenize)),
        ('tfidf',TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [6]:
pipeline.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
def classification_rep_col(y_test, y_pred):
    for i, col in enumerate(y_test.columns): 
        print('-------:',col,':-------')
        print(classification_report(y_test.iloc[:,i], y_pred[:,i]))

In [8]:
y_pred = pipeline.predict(X_test)

In [9]:
classification_rep_col(y_test, y_pred)

-------: related :-------
             precision    recall  f1-score   support

          0       0.60      0.38      0.46      1491
          1       0.83      0.93      0.87      5012
          2       0.60      0.12      0.20        51

avg / total       0.77      0.79      0.78      6554

-------: request :-------
             precision    recall  f1-score   support

          0       0.88      0.98      0.93      5422
          1       0.83      0.36      0.50      1132

avg / total       0.87      0.88      0.86      6554

-------: offer :-------
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6523
          1       0.00      0.00      0.00        31

avg / total       0.99      1.00      0.99      6554

-------: aid_related :-------
             precision    recall  f1-score   support

          0       0.72      0.87      0.79      3842
          1       0.74      0.52      0.61      2712

avg / total       0.73      0.73   

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [10]:
parameters = {
    'tfidf__norm':['l2','l1'],
    'clf__estimator__min_samples_split':[2,3],
    }

grid_clf = GridSearchCV(pipeline, parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [11]:
grid_clf.fit(X_train,y_train)
y_pred = grid_clf.predict(X_test)

In [12]:
print(grid_clf.best_params_)

{'clf__estimator__min_samples_split': 2, 'tfidf__norm': 'l2'}


In [13]:
accuracy = (y_pred == y_test).sum()/y_test.shape[0]
print('accuracy :', accuracy)
classification_rep_col(y_test, y_pred)

accuracy : related                   0.796308
request                   0.879463
offer                     0.995270
aid_related               0.734666
medical_help              0.922185
medical_products          0.951480
search_and_rescue         0.974367
security                  0.980012
military                  0.966585
child_alone               1.000000
water                     0.942173
food                      0.925236
shelter                   0.924931
clothing                  0.985047
money                     0.974977
missing_people            0.988557
refugees                  0.968416
death                     0.958956
other_aid                 0.869393
infrastructure_related    0.934696
transport                 0.956057
buildings                 0.953769
electricity               0.978944
tools                     0.992676
hospitals                 0.988862
shops                     0.994507
aid_centers               0.988404
other_infrastructure      0.955447
weather_r

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [8]:
# using AdaBoost
pipeline_ada = Pipeline([
        ('vect',CountVectorizer(tokenizer = tokenize)),
        ('tfidf',TfidfTransformer()),
        ('clf', MultiOutputClassifier(AdaBoostClassifier(n_estimators=50, learning_rate = 1)))
])

# using KNeighborsClassifier
pipeline_kneighbors = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', KNeighborsClassifier())
])

In [15]:
pipeline_ada.fit(X_train, y_train)
y_pred = pipeline_ada.predict(X_test)
classification_rep_col(y_test, y_pred)

-------: related :-------
             precision    recall  f1-score   support

          0       0.57      0.26      0.35      1491
          1       0.80      0.94      0.87      5012
          2       0.35      0.12      0.18        51

avg / total       0.75      0.78      0.75      6554

-------: request :-------
             precision    recall  f1-score   support

          0       0.91      0.97      0.94      5422
          1       0.77      0.52      0.62      1132

avg / total       0.88      0.89      0.88      6554

-------: offer :-------
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6523
          1       0.00      0.00      0.00        31

avg / total       0.99      0.99      0.99      6554

-------: aid_related :-------
             precision    recall  f1-score   support

          0       0.76      0.86      0.81      3842
          1       0.76      0.61      0.67      2712

avg / total       0.76      0.76   

In [18]:
pipeline_kneighbors.fit(X_train, y_train)
y_pred = pipeline_kneighbors.predict(X_test)
classification_rep_col(y_test, y_pred)

-------: related :-------
             precision    recall  f1-score   support

          0       0.60      0.32      0.42      1491
          1       0.82      0.93      0.87      5012
          2       0.33      0.33      0.33        51

avg / total       0.76      0.79      0.76      6554

-------: request :-------
             precision    recall  f1-score   support

          0       0.84      1.00      0.91      5422
          1       0.84      0.07      0.13      1132

avg / total       0.84      0.84      0.78      6554

-------: offer :-------
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6523
          1       0.00      0.00      0.00        31

avg / total       0.99      1.00      0.99      6554

-------: aid_related :-------
             precision    recall  f1-score   support

          0       0.60      1.00      0.75      3842
          1       0.86      0.04      0.08      2712

avg / total       0.70      0.60   

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Comparing the classification reports of the different models, we choose to go AdaBoostClassifier with as our final model.

These were compared on the basis of the macro average F1 score obtained by all the 3 models. The avg/total F1 scores were compared across all the classes and the highest average F1 score was chosen.

Now we will use gridsearch on AdaBoostClassifier to optimize it.

In [17]:
parameters = {
    'vect__stop_words': ['english',None],
    'tfidf__use_idf' :[True, False],
    'clf__estimator__n_estimators' : [60,70],
    'clf__estimator__learning_rate' :[1,0.5,2],
    }
grid_clf = GridSearchCV(pipeline_ada, parameters,verbose=2)

In [18]:
grid_clf.fit(X_train, y_train)

Fitting 3 folds for each of 24 candidates, totalling 72 fits
[CV] clf__estimator__learning_rate=1, clf__estimator__n_estimators=60, tfidf__use_idf=True, vect__stop_words=english 
[CV]  clf__estimator__learning_rate=1, clf__estimator__n_estimators=60, tfidf__use_idf=True, vect__stop_words=english - 1.7min
[CV] clf__estimator__learning_rate=1, clf__estimator__n_estimators=60, tfidf__use_idf=True, vect__stop_words=english 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.7min remaining:    0.0s


[CV]  clf__estimator__learning_rate=1, clf__estimator__n_estimators=60, tfidf__use_idf=True, vect__stop_words=english - 1.8min
[CV] clf__estimator__learning_rate=1, clf__estimator__n_estimators=60, tfidf__use_idf=True, vect__stop_words=english 
[CV]  clf__estimator__learning_rate=1, clf__estimator__n_estimators=60, tfidf__use_idf=True, vect__stop_words=english - 1.9min
[CV] clf__estimator__learning_rate=1, clf__estimator__n_estimators=60, tfidf__use_idf=True, vect__stop_words=None 
[CV]  clf__estimator__learning_rate=1, clf__estimator__n_estimators=60, tfidf__use_idf=True, vect__stop_words=None - 2.8min
[CV] clf__estimator__learning_rate=1, clf__estimator__n_estimators=60, tfidf__use_idf=True, vect__stop_words=None 
[CV]  clf__estimator__learning_rate=1, clf__estimator__n_estimators=60, tfidf__use_idf=True, vect__stop_words=None - 2.6min
[CV] clf__estimator__learning_rate=1, clf__estimator__n_estimators=60, tfidf__use_idf=True, vect__stop_words=None 
[CV]  clf__estimator__learning_rate

[CV]  clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=60, tfidf__use_idf=False, vect__stop_words=None - 2.0min
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=70, tfidf__use_idf=True, vect__stop_words=english 
[CV]  clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=70, tfidf__use_idf=True, vect__stop_words=english - 1.8min
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=70, tfidf__use_idf=True, vect__stop_words=english 
[CV]  clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=70, tfidf__use_idf=True, vect__stop_words=english - 1.8min
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=70, tfidf__use_idf=True, vect__stop_words=english 
[CV]  clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=70, tfidf__use_idf=True, vect__stop_words=english - 1.8min
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=70, tfidf__use_idf=True, vect__stop_words=None 
[CV]  clf

[CV]  clf__estimator__learning_rate=2, clf__estimator__n_estimators=70, tfidf__use_idf=False, vect__stop_words=None - 2.3min
[CV] clf__estimator__learning_rate=2, clf__estimator__n_estimators=70, tfidf__use_idf=False, vect__stop_words=None 
[CV]  clf__estimator__learning_rate=2, clf__estimator__n_estimators=70, tfidf__use_idf=False, vect__stop_words=None - 2.3min
[CV] clf__estimator__learning_rate=2, clf__estimator__n_estimators=70, tfidf__use_idf=False, vect__stop_words=None 
[CV]  clf__estimator__learning_rate=2, clf__estimator__n_estimators=70, tfidf__use_idf=False, vect__stop_words=None - 2.3min


[Parallel(n_jobs=1)]: Done  72 out of  72 | elapsed: 149.0min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...timator=None, learning_rate=1,
          n_estimators=50, random_state=None),
           n_jobs=1))]),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'vect__stop_words': ['english', None], 'tfidf__use_idf': [True, False], 'clf__estimator__n_estimators': [60, 70], 'clf__estimator__learning_rate': [1, 0.5, 2]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=2)

In [19]:
print(grid_clf.best_params_)

{'clf__estimator__learning_rate': 0.5, 'clf__estimator__n_estimators': 70, 'tfidf__use_idf': True, 'vect__stop_words': None}


### 9. Export your model as a pickle file

In [20]:
joblib.dump(grid_clf.best_estimator_,'ada_grid.pkl')

['ada_grid.pkl']

In [21]:
pipe = joblib.load('ada_grid.pkl')

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.